# JeopardyRush!

### Cleaning the Jeopardy! question data and converting it to JSON

We do this to make it easier to work with the data in Javascript for our application

First we import the stuff we need, including NLTK, a natural langauge processinig library, to clean up the answer data a bit more so we can accept more answers from the user. Then we'll read in the data as a dataframe.

In [80]:
import pandas as pd

# Just to get rid of the annoying warnings in the notebook
import warnings
warnings.filterwarnings('ignore')
import re

In [81]:
jeo_df = pd.read_csv('./JEOPARDY_CSV.csv')

In [82]:
jeo_df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


First I want to make the column names more usable by eliminating white space. Then I'll check for null values and see how to best deal with them

In [83]:
jeo_df.columns

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')

In [84]:
jeo_df.columns = ['show_number', 'air_date', 'round', 'category', 'value', 'question', 'answer']

Technically, some Jeopardy enthusiasts might be upset we have the question as the answer and vice versa, but we'll let it slide.

In [85]:
jeo_df.isnull().sum()

show_number    0
air_date       0
round          0
category       0
value          0
question       0
answer         2
dtype: int64

The data is pretty clean, but let's see what's going on with those two null answer values

In [86]:
jeo_df[jeo_df['answer'].isnull()]

,show_number,air_date,round,category,value,question,answer
94817,4346,2003-06-23,Jeopardy!,"GOING ""N""SANE",$200,"It often precedes ""and void""",NaN
143297,6177,2011-06-21,Double Jeopardy!,NOTHING,$400,"This word for ""nothing"" precedes ""and void"" to...",NaN


Hilarious! The actual answers are "null", so when whoever wrangled the data converted the null values, these answers turned into NaN. Let's turn them back.

In [87]:
jeo_df.fillna('null', inplace=True)

We want to use the value in our application, but Final Jeopardy questions don't have a value. These are great questions, so I'd like to use them. Let's convert "None" in the value column to $2000, as Final Jeopardy questions are difficult.

In [88]:
jeo_df['value'].replace('None', '$2,000', inplace=True)

We're going to want to use the value as both as string to display and an integer to keep track of scores, so let's create a new column with the value as an integer

In [89]:
jeo_df['value_int'] = jeo_df['value'].apply(lambda x: int(x.replace('$','').replace(',','')))

I might want to work with the show number at some point, but the air date and round I don't think we need, so let's drop them.

In [90]:
jeo_df.drop(columns=['air_date', 'round'], inplace=True)

In [91]:
jeo_df.shape

(216930, 6)

We're going to transform the data a bit, removing video and photo clues as that media is not in our data to be displayed to the user. We don't want "name this flower seen above" in our questions if we don't have a photo to render.

In [92]:
# I wanted to see if I could extract the jpg images from the questions if they are present, so we can keep the
# photo clues in the data. But, unfortunately, most of them are no longer being hosted, so it's best to drop them
# However, this is how I extracted the jpg urls using a regular expression

def extract_media(question):
    jpg = ''
    if '.jpg' in question:
        urls = re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+', question)
        for url in urls:
            if 'jpg' in url:
                jpg = url
            else:
                continue
        return jpg
    else:
        return 0
    
jeo_df['media'] = jeo_df['question'].apply(lambda x: extract_media(x))

In [93]:
# Remove photo, video and audio clues which tend to have the word "here" in them, as in "seen here" or "heard here"
jeo_df = jeo_df[~jeo_df['question'].str.contains(' here ')]
jeo_df = jeo_df[jeo_df['question'].apply(lambda x: ('seen' not in x.lower() and 'video' not in x.lower() and 'pictured' not in x.lower() and 'shown' not in x.lower()))]

In [94]:
# Replace ampersand with the more natural 'and' for matching later
jeo_df['answer'] = jeo_df['answer'].str.replace('&', 'and')

In [95]:
# Parentheses are tricky in the answers, so best to remove the answers containing them
jeo_df = jeo_df[~jeo_df['answer'].str.contains('\(')]

In [96]:
jeo_df = jeo_df[~jeo_df['answer'].str.contains('\/')]

In [97]:
# Quotes we can simply remove
jeo_df['answer'] = jeo_df['answer'].str.replace('"','')

In [98]:
# Removing the answers with commas will be helpful, as well
jeo_df = jeo_df[~jeo_df['answer'].str.contains(',')]

In [99]:
# removing the and a at the beginning of answers
jeo_df['the_or_a'] = jeo_df['answer'].apply(lambda x: 'yes' if x.lower().startswith('the ') \
                                            or x.lower().startswith('a ') or x.lower().startswith('an ') else 'no')
jeo_df = jeo_df[jeo_df['the_or_a'] == 'no']

In [100]:
# Removing answers with three or more words to simplify things
jeo_df['long_answer'] = jeo_df.answer.apply(lambda x: 1 if len(x.split()) > 2 else 0)
jeo_df = jeo_df[jeo_df['long_answer'] == 0]

In [101]:
def acceptable_answer(answer):
    """New column with last name as another acceptable answer for questions about people"""
    
    answer_list = [answer]
    
    num_map = {'0': 'zero', '1' : 'one', '2': 'two', '3': 'three', '4': 'four', '5': 'five', '6': 'six', '7': 'seven',\
               '8': 'eight', '9': 'nine', '10': 'ten'}
    
    if answer in num_map:
        answer_list.append(num_map[answer])
    
    split_up = answer.split()

    if len(split_up) == 2:
        if (split_up[0][0].isupper() and split_up[1][0].isupper()):
            answer_list.append(split_up[1])
            
    return answer_list
    

In [102]:
jeo_df['acceptable_answer'] = jeo_df.answer.apply(lambda x: acceptable_answer(x))

In [103]:
# Getting rid of the columns we don't need
jeo_df = jeo_df[['category', 'value', 'question', 'value_int', 'acceptable_answer']]

I think we're ready to convert this to JSON so we can access it through Javascript

In [105]:
jeo_df

,category,value,question,value_int,acceptable_answer
0,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",200,[Copernicus]
1,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,200,"[Jim Thorpe, Thorpe]"
2,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,200,[Arizona]
3,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",200,[McDonald's]
4,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",200,"[John Adams, Adams]"
...,...,...,...,...,...
216923,WORLD CAPITALS,$2000,It's on the Suriname River,2000,[Paramaribo]
216924,OFF-BROADWAY,$2000,In 2006 the cast of this long-running hit emba...,2000,[Stomp]
216925,RIDDLE ME THIS,$2000,This Puccini opera turns on the solution to 3 ...,2000,[Turandot]
216927,AUTHORS IN THEIR YOUTH,$2000,"In Penny Lane, where this ""Hellraiser"" grew up...",2000,"[Clive Barker, Barker]"


In [49]:
jeo_df.sample(10000).to_json('jeopardy_data.json', orient='records')